# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import Dataset

In [2]:
df = pd.read_csv(r'./lifestyle_sustainability_data.csv',index_col='ParticipantID')
df.head(1)

,Age,Location,DietType,LocalFoodFrequency,TransportationMode,EnergySource,HomeType,HomeSize,ClothingFrequency,SustainableBrands,EnvironmentalAwareness,CommunityInvolvement,MonthlyElectricityConsumption,MonthlyWaterConsumption,Gender,UsingPlasticProducts,DisposalMethods,PhysicalActivities,Rating
ParticipantID,,,,,,,,,,,,,,,,,,,
1,35,Urban,Mostly Plant-Based,Often,Bike,Renewable,Apartment,800,Rarely,True,5,High,100,1500,Female,Rarely,Composting,High,5


In [3]:
df.MonthlyWaterConsumption.mean()
# df.set_option('display.max')

3139.521042084168

In [4]:
# Import EDA library

In [5]:
import sweetviz as sv

/opt/miniconda3/envs/ml_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Create the report
my_report = sv.analyze(df)

# Display the report in an HTML file
my_report.show_html("EDA_Report.html")

Done! Use 'show' commands to display/save.   |██████████| [100%]   00:01 -> (00:00 left)


Report EDA_Report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


# Data Preprocessing 

### segregrate data into continuos , nominal  and ordinal
### create order for ordinal data

In [7]:
continuous_columns = ['Age','HomeSize','MonthlyElectricityConsumption','MonthlyWaterConsumption']
nominal_columns = ['Location', 'DietType','TransportationMode','SustainableBrands','Gender','DisposalMethods' ]
ordinal_columns = ['LocalFoodFrequency','EnergySource','ClothingFrequency','EnvironmentalAwareness','CommunityInvolvement','UsingPlasticProducts','PhysicalActivities']

In [8]:
order = [['Rarely','Sometimes','Often','Always'],
       ['Renewable','Mixed','Non-Renewable',],
       ['Rarely', 'Sometimes', 'Often', 'Always'],
       [1,2,3,4,5],
       ['Low','Moderate','High'],
       ['Never','Rarely','Sometimes','Often'],
       ['Low','Moderate','High']]

# Pipeline and Imputation
#### CommunityInvolvement and PhysicalActivities are having missing values

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder,OrdinalEncoder

In [10]:
# continuous_transformer = Pipeline([("Imputer",SimpleImputer(strategy='mean')),("step 2 ",StandardScaler(),)])
# nominal_transformer = Pipeline([('S Imputer', SimpleImputer(strategy='most_frequent'),),('sstep 2', OneHotEncoder(handle_unknown='ignore'),)])
# ordinal_transformer = Pipeline([('S Imputer',SimpleImputer(strategy='most_frequent'),), ('step 2',OrdinalEncoder(categories = order),)])


In [11]:
continuous_transformer = Pipeline([("step 2 ",StandardScaler(),)])
nominal_transformer = Pipeline([('sstep 2', OneHotEncoder(handle_unknown='ignore'),)])
ordinal_transformer = Pipeline([('S Imputer',SimpleImputer(strategy='most_frequent'),), ('step 2',OrdinalEncoder(categories = order),)])


# Column Transformers

In [12]:
from sklearn.compose import ColumnTransformer

In [13]:
preprocess = ColumnTransformer([("Continuous Data",continuous_transformer,continuous_columns),
                               ("Nominal Data", nominal_transformer,nominal_columns),
                               ("Ordinal Data", ordinal_transformer,ordinal_columns)])
preprocess

ColumnTransformer(transformers=[('Continuous Data',
                                 Pipeline(steps=[('step 2 ',
                                                  StandardScaler())]),
                                 ['Age', 'HomeSize',
                                  'MonthlyElectricityConsumption',
                                  'MonthlyWaterConsumption']),
                                ('Nominal Data',
                                 Pipeline(steps=[('sstep 2',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['Location', 'DietType', 'TransportationMode',
                                  'SustainableBrands', 'Gender',
                                  'DisposalMethods']),
                                (...
                                                                              'Often',
                                                                              'Always'],
                                                                             ['Renewable',
                                                                              'Mixed',
                                                                              'Non-Renewable'],
                                                                             ['Rarely',
                                                                              'Sometimes',
                                                                              'Often',
                                                                              'Always'],
                                                                             [1,
                                                                              2,
                                                                              3,
                                                                              4,
                                                                              5],
                                                                             ['Low',
                                                                              'Moderate',
                                                                              'High'],
                                                                             ['Never',
                                                                              'Rarely',
                                                                              'Sometimes',
                                                                              'Often'],
                                                                             ['Low',
                                                                              'Moderate',
                                                                              'High']]))]),
                                 ['LocalFoodFrequency', 'EnergySource',
                                  'ClothingFrequency', 'EnvironmentalAwareness',
                                  'CommunityInvolvement',
                                  'UsingPlasticProducts',
                                  'PhysicalActivities'])])

# Model Building using pipeline

1. Logistics Regression
2. SVC
3. KNN Classfication
4. Decision Tree
5. Naive Bayes

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

# Model Training

In [15]:
X = df.drop(['Rating'] , axis =1 )
Y = df['Rating']

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=23)


allmodels = [LogisticRegression(),SVC(),KNeighborsClassifier(),DecisionTreeClassifier(),GaussianNB()]
# models = [ Pipeline([("DataPreprocessing",preprocess),("Algorithm",ml_model)]) for ml_model in allmodels]

accuracy_score_all_models = {}
for ml_model in allmodels:
    trained_model = Pipeline([("DataPreprocessing",preprocess),("Algorithm",ml_model)])
    trained_model.fit(X_train,Y_train)
    Y_predict = trained_model.predict(X_test)
    score = accuracy_score(Y_test, Y_predict)
    accuracy_score_all_models[ml_model] = score

# for (k,v)  in accuracy_score_all_models:
#     print(f"{k} : {v}")

accuracy_score_all_models
    
# model = Pipeline([("DataPreprocessing",preprocess),("Algorithm",LogisticRegression())])
# model.fit(X_train,Y_train)
# Y_predict = model.predict(X_test)
# accuracy_score(Y_test, Y_predict)

{LogisticRegression(): 0.65,
 SVC(): 0.71,
 KNeighborsClassifier(): 0.68,
 DecisionTreeClassifier(): 0.66,
 GaussianNB(): 0.64}

# SVC has highest accuracy So Lets use SVC algorithm .

In [18]:
knn = Pipeline([("DataPreprocessing",preprocess),("Algorithm",KNeighborsClassifier())])
knn.fit(X_train,Y_train)



Pipeline(steps=[('DataPreprocessing',
                 ColumnTransformer(transformers=[('Continuous Data',
                                                  Pipeline(steps=[('step 2 ',
                                                                   StandardScaler())]),
                                                  ['Age', 'HomeSize',
                                                   'MonthlyElectricityConsumption',
                                                   'MonthlyWaterConsumption']),
                                                 ('Nominal Data',
                                                  Pipeline(steps=[('sstep 2',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Location', 'DietType',
                                                   'TransportationMode',
                                                   'SustainableB...
                                                                                               'Non-Renewable'],
                                                                                              ['Rarely',
                                                                                               'Sometimes',
                                                                                               'Often',
                                                                                               'Always'],
                                                                                              [1,
                                                                                               2,
                                                                                               3,
                                                                                               4,
                                                                                               5],
                                                                                              ['Low',
                                                                                               'Moderate',
                                                                                               'High'],
                                                                                              ['Never',
                                                                                               'Rarely',
                                                                                               'Sometimes',
                                                                                               'Often'],
                                                                                              ['Low',
                                                                                               'Moderate',
                                                                                               'High']]))]),
                                                  ['LocalFoodFrequency',
                                                   'EnergySource',
                                                   'ClothingFrequency',
                                                   'EnvironmentalAwareness',
                                                   'CommunityInvolvement',
                                                   'UsingPlasticProducts',
                                                   'PhysicalActivities'])])),
                ('Algorithm', KNeighborsClassifier())])

In [19]:
import pickle


In [20]:
pickle.dump(knn, open("sustainability_rating_predictor.pkl", 'wb'))

In [21]:
# x = '''Age	Location	DietType	LocalFoodFrequency	TransportationMode	EnergySource	HomeType	HomeSize	ClothingFrequency	SustainableBrands	EnvironmentalAwareness	CommunityInvolvement	MonthlyElectricityConsumption	MonthlyWaterConsumption	Gender	UsingPlasticProducts	DisposalMethods	PhysicalActivities	Rating'''

# y = []
# for  col in x.split():
#     if col in continuous_columns:
#         y.append(f'{col} = st.number_input("Enter {col}") ')
#         # print(y)
#     else:
#         y.append(f'{col} = st.selectbox("pick {col}", {list(df[col].unique())})')

# # y = "\n".join([f'{i} = st.selectbox("Pick a {i}",{foo(i)})' for i in x.split()])

# z = '\n'.join(y)

# with open('./code.txt','w') as file:
#     file.write(z)

In [22]:
y

NameError: name 'y' is not defined

In [ ]:
print("\n".join(['1','2','3']))

1
2
3
